In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
import websocket  # NOTE: websocket-client (https://github.com/websocket-client/websocket-client)
import uuid
import json
import urllib.request
import urllib.parse
from PIL import Image
import io
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import requests

In [112]:
def post_image(image_path, subfolder, folder_type):

    url = f"http://{server_address}/upload/image"

    # Additional Form Data
    data = {
        "overwrite": "true",                 # Overwrite behavior (optional)
        "type": folder_type,                 # Directory type: input, temp, or output
        "subfolder": subfolder               # Subfolder name (optional)
    }

    files = {
        "image": open(image_path, "rb")       # Image file
    }

    try:
        # Send POST Request
        response = requests.post(url, data=data, files=files)

        # Handle Response
        if response.status_code == 200:
            print("Image uploaded successfully!")
            print("Response JSON:", response.json())
        else:
            print(f"Failed to upload image. Status Code: {response.status_code}")
            print("Response Text:", response.text)

            return None

        return response.json()

    except Exception as e:
        print("Error occurred:", e)


def queue_prompt(prompt):

    url = f"http://{server_address}/prompt"

    try:
        # Prepare the payload
        payload = {"prompt": prompt, "client_id": client_id}

        # Send the POST request using the requests library
        response = requests.post(f"http://{server_address}/prompt", json=payload)

        # Raise an error for bad status codes
        response.raise_for_status()

        # Parse and return the JSON response
        return response.json()

    except requests.RequestException as e:
        print(f"Error in queue_prompt: {e}")
        return None


def get_history(prompt_id):

    url = f"http://{server_address}/history/{prompt_id}"

    try:
        # Send the GET request using the requests library
        response = requests.get(url)

        # Raise an error for bad status codes
        response.raise_for_status()

        # Parse and return the JSON response
        return response.json()

    except requests.RequestException as e:
        print(f"Error in get_history: {e}")
        return None


def get_image(filename, subfolder, folder_type):
    url = f"http://{server_address}/view"
    params = {
        "filename": filename,
        "subfolder": subfolder,
        "type": folder_type,
    }

    try:
        # Send the GET request with query parameters
        response = requests.get(url, params=params)

        # Raise an error for bad status codes
        response.raise_for_status()

        # Return the raw content (binary data)
        return response.content

    except requests.RequestException as e:
        print(f"Error in get_image: {e}")
        return None

In [118]:
def edit_hair(src_path, ref_path, ws):


    "-----------------------Upload Images-----------------------------"
    src = post_image(src_path, subfolder, folder_type)
    ref = post_image(ref_path, subfolder, folder_type)


    "-----------------------Execute Workflow-----------------------------"
    # Load the workflow JSON
    with open(workflow_api_path, "r", encoding="utf-8") as f:
        workflow_data = f.read()

    prompt = json.loads(workflow_data)

    src_image = f"{comfyUI}/{src['type']}/{src['subfolder']}/{src['name']}"
    ref_image = f"{comfyUI}/{ref['type']}/{ref['subfolder']}/{ref['name']}"

    # Set the text prompt and seed
    prompt["1"]["inputs"]["image"] = src_image
    prompt["2"]["inputs"]["image"] = ref_image

    prompt_response = queue_prompt(prompt)
    prompt_id = prompt_response['prompt_id']

    if not prompt_response:
        print("Failed to queue the prompt.")
    else:
        print("Prompt queued successfully!")
        print("Prompt ID:", prompt_id)


    "-----------------------Waiting for Execute Finish------------------------"
    while True:
        out = ws.recv()
        if isinstance(out, str):
            message = json.loads(out)
            if message['type'] == 'executing':
                data = message['data']
                if data['node'] is None and data['prompt_id'] == prompt_id:
                    break #Execution is done
        else:
            # If you want to be able to decode the binary stream for latent previews, here is how you can do it:
            # bytesIO = BytesIO(out[8:])
            # preview_image = Image.open(bytesIO) # This is your preview in PIL image format, store it in a global
            continue #previews are binary data


    "-----------------------Get Output-----------------------------"
    history = get_history(prompt_id)
    history_data = history[prompt_id]

    for node_id in history_data['outputs']:
        node_output = history_data['outputs'][node_id]
        if 'images' in node_output:
            for image in node_output['images']:
                if image['type'] == 'output':
                    image_data = get_image(image['filename'], image['subfolder'], image['type'])

                    # Save the binary data to a file
                    if image_data:
                        output_path = f"{output_dir}/{image['filename']}"
                        # Convert binary data to image
                        img = Image.open(io.BytesIO(image_data))

                        # Save the image as JPG (you can specify a different path if needed)
                        img.save(output_path, "JPEG")

                        print(f"Image saved as {output_path}")
                    else:
                        print(f"Failed to retrieve the image: {image['filename']}")

In [119]:
server_address = "rnytu-34-105-18-165.a.free.pinggy.link:80"
client_id = str(uuid.uuid4())

comfyUI = "/kaggle/working/ComfyUI/"
workflow_api_path = "/content/drive/MyDrive/ỨngDụngvàXửLíẢnh/Final/Workflow/Virtual_TryOn_api_PYTHONRUN.json"

src_path = "/content/drive/MyDrive/ỨngDụngvàXửLíẢnh/Final/data/Men/360° メンズヘアカタログ 長さ別検索.jpg"
ref_path = "/content/drive/MyDrive/ỨngDụngvàXửLíẢnh/Final/data/Men/[Gallery]最強ヘアカタログ、ショート10スタイル.jpg"
subfolder = "UPLOADED_IMAGES"
folder_type = "input"

output_dir = "/content/drive/MyDrive/ỨngDụngvàXửLíẢnh/Final/Output"


In [120]:
ws = websocket.WebSocket()
ws.connect(f"ws://{server_address}/ws?clientId={client_id}")
edit_hair(src_path, ref_path,ws)
ws.close()

Image uploaded successfully!
Response JSON: {'name': '360° メンズヘアカタログ 長さ別検索.jpg', 'subfolder': 'UPLOADED_IMAGES', 'type': 'input'}
Image uploaded successfully!
Response JSON: {'name': '[Gallery]最強ヘアカタログ、ショート10スタイル.jpg', 'subfolder': 'UPLOADED_IMAGES', 'type': 'input'}
Prompt queued successfully!
Prompt ID: c9ac3f33-a060-4dcb-8452-bc3b2f716b57
Image saved as /content/drive/MyDrive/ỨngDụngvàXửLíẢnh/Final/Output/ComfyUI_00009_.png
